In [88]:
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()

In [89]:
tf.reset_default_graph()

In [91]:
x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [92]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2,10]), name='weight')
    B1 = tf.Variable(tf.random_normal([10]), name='bias')
    L1 = tf.sigmoid(tf.matmul(X, W1) + B1)
    
    w1_hist = tf.summary.histogram("w1", W1)
    b1_hist = tf.summary.histogram("b1", B1)
    l1_hist = tf.summary.histogram("l1", L1)

with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([10,1]), name='weight2')
    B2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(L1, W2) + B2)

    w2_hist = tf.summary.histogram("w2", W2)
    b2_hist = tf.summary.histogram("b2", B2)

In [94]:
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Accuracy
# H가 0.5 이상이면 true else false
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

with tf.Session() as sess:
    # Add Graph
    merged_summary = tf.summary.merge_all()

    writer = tf.summary.FileWriter('./logs')
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step = step)

        if step % 1000 == 0:
            print(step, cost_val)

    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("h:", h, "c:", c, "a:", a)

0 0.8316715
1000 0.49852318
2000 0.14774734
3000 0.05518782
4000 0.030073727
5000 0.019815003
6000 0.014489699
7000 0.011298601
8000 0.009198439
9000 0.007722643
10000 0.006634321
h: [[0.00531756]
 [0.9927032 ]
 [0.99442875]
 [0.00825712]] c: [[0.]
 [1.]
 [1.]
 [0.]] a: 1.0
